# IPO Prospectus Data Exploration and Analysis

## Goal
Use the IPO Prospectus data to create a model that predicts whether a company will have a successful IPO or not.

## Data

Acquired from scraping thousands of SEC prospectus documents. The data was stored in HTML format, and then converted to a CSV file. Things like keywords, underwriting, and financial data were extracted from the HTML and stored in new columns. 


## Preprocessing Data

Applying the fundamental rules of data cleaning and feature engineering to the data.

In [43]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
warnings.filterwarnings('ignore')

def create_original_dataframe():
    original_df = pd.read_csv('../data/all_financial_with_keywords.csv') 
    original_df = original_df.drop_duplicates()
    return original_df

original_df = create_original_dataframe()

### Prep Target Column
def prep_target_variable(original_df, column='diff'):
    # Convert to percent based deltas
    original_df['open'] = original_df[column] + original_df['close']
    original_df['diff_percent'] = original_df[column] / original_df['close']
    
   # Remove the over 100% change values
    final_df = original_df[(original_df['diff_percent'] > -1.0) & (original_df['diff_percent'] < 1.0)]
    return final_df

original_df = prep_target_variable(original_df)
target_column = 'diff_percent'

bank_columns = [
    'citigroup', 'citi', 'morgan stanley', 'ubs', 'barclays', 'wells fargo', 
    'goldman sachs', 'deutsche bank', 'credit suisse', 'merrill lynch', 
    'rbc capital', 'jefferies', 'stifel', 'morgan stanley.1', 'merrill lynch.1', 
    'goldman sachs.1', 'raymond james', 'piper jaffray', 'robert w. baird', 
    'william blair', 'william blair.1', 'hill road', 'document_length', 'word_count'
]

keyword_columns = [
    'technology', 'software', 'ai', 'machine learning', 'cloud', 'saas', 
    'platform', 'digital', 'data', 'analytics', 'algorithm', 'automation', 
    'blockchain', 'cryptocurrency', 'cybersecurity', 'subscription', 'recurring',
    'e-commerce', 'mobile', 'app', 'virtual', 'healthcare', 'biotech', 
    'pharmaceutical', 'medical', 'clinical', 'energy', 'renewable', 'solar', 
    'electric', 'battery', 'real estate', 'logistics', 'transportation', 
    'automotive', 'document_length', 'word_count' 
]

financial_columns = [
    'additional_paid_in_capital_trend', 'additional_paid_in_capital_recent',
    'total_assets_trend', 'total_assets_recent', 'total_current_liabilities_trend', 
    'total_current_liabilities_recent', 'total_liabilities_trend', 'total_liabilities_recent', 
    'cash_trend', 'cash_recent', 'total_capitalization_trend', 'total_capitalization_recent', 
    'total_current_assets_trend', 'total_current_assets_recent', 'volume', 
    'close', 'diff', 'price_public_total', 'public_price_per_share_x', 
    'public_price_per_share_y' 
]

In [44]:
import numpy as np

def prepare_bank_count_feature(df, columns):
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    ### Convert NA to 0
    df[columns] = df[columns].fillna(0)

    ### Remove Dupes
    dupes = [
        'morgan stanley.1',
        'merrill lynch.1',
        'goldman sachs.1',
        'william blair.1'
    ]

    for dupe in dupes:
        # Fix: Check if column exists in the subset
        if dupe not in df[columns].columns:
            continue
        root_name = dupe.split('.')[0]
        # Fix: Work directly with the main dataframe
        df[root_name] = df[root_name] + df[dupe]
        df = df.drop(dupe, axis=1)
        # Update columns list to reflect dropped column
        if dupe in columns:
            columns = [col for col in columns if col != dupe]
        
    ### Feature Engineer Bank Group Column
    cols = list(df[columns].columns)
    # Fix: Add new column to main dataframe, not subset
    df['bank_group'] = df[columns].apply(
        lambda row: [col for col, val in row.items() 
                    if val >= 1 and col != 'word_count' and col != 'document_length'], 
        axis=1
    )
    # Encoded value of bank name is its index in the cols list 
    df['bank_group'] = df['bank_group'].apply(
        lambda row: [cols.index(bank_name) for bank_name in row]
    )

    ### Feature Engineer Unique Bank Count
    df['unique_bank_count'] = df['bank_group'].apply(lambda row: len(row))

    ### Feature Engineer Bank Count Ratios
    for col in df[columns].columns:
        if col in ['bank_group', 'word_count', 'document_length', 'unique_bank_count']:
            continue
        new_col = f'{col}-word-count-ratio'
        df[new_col] = df[col] / df['word_count']

    ### Feature Engineer Parallel Log1p Column
    for col in df[columns].columns:
        if col in ['bank_group', 'word_count', 'document_length', 'unique_bank_count'] or 'word-count-ratio' in col:
            continue
        new_col = f'{col}-log1p'
        df[new_col] = np.log1p(df[col])   

    return df


import numpy as np

def prepare_keyword_count_feature(df, columns):
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    ### Convert NA to 0
    df[columns] = df[columns].fillna(0)
        
    ### Feature Engineer Keyword Group Column
    cols = list(df[columns].columns)
    # Remove non-keyword columns, check if column has a value
    df['keyword_group'] = df[columns].apply(
        lambda row: [col for col, val in row.items() 
                    if val >= 1 and col != 'word_count' and col != 'document_length'], 
        axis=1
    )
    # Encoded value of keyword name is its index in the cols list 
    df['keyword_group'] = df['keyword_group'].apply(
        lambda row: [cols.index(keyword_name) for keyword_name in row]
    )

    ### Feature Engineer Unique Keyword Count
    # Count the length of the keyword_groups array
    df['unique_keyword_count'] = df['keyword_group'].apply(lambda row: len(row))

    ### Feature Engineer Keyword Count Ratios
    for col in df[columns].columns:
        if col in ['keyword_group', 'word_count', 'document_length', 'unique_keyword_count']:
            continue
        new_col = f'{col}-word-count-ratio'
        # Fix: Reference the main dataframe for both numerator and denominator
        df[new_col] = df[col] / df['word_count']

    ### Feature Engineer Parallel Log1p Column
    for col in df[columns].columns:
        if col in ['keyword_group', 'word_count', 'document_length', 'unique_keyword_count'] or 'word-count-ratio' in col:
            continue
        new_col = f'{col}-log1p'
        # Fix: Reference the main dataframe
        df[new_col] = np.log1p(df[col])   

    return df


def prepare_financial_feature(df, columns):
    ### Calculate computed columns
    def safe_ratio(numerator, denominator, fill_value=np.nan):
        return np.where(denominator != 0, numerator / denominator, fill_value)

    # Make a copy to avoid modifying the original
    df = df.copy()
    
    ### Remove dupes and unused columns
    # Work directly with the main dataframe for structural changes
    df = df.drop("public_price_per_share_x", axis=1, errors="ignore")
    df = df.rename(columns={"public_price_per_share_y": "public_price_per_share"})
    df = df.drop("close", axis=1, errors="ignore")
    df = df.drop("volume", axis=1, errors="ignore")
    
    # Update columns list to reflect any dropped columns
    columns = [col for col in columns if col in df.columns]
    # Add the renamed column if it exists
    if "public_price_per_share" in df.columns and "public_price_per_share" not in columns:
        columns.append("public_price_per_share")

    ### Normalize
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[columns])
    # Fix: Assign back to the main dataframe with proper index alignment
    df[columns] = pd.DataFrame(scaled_data, columns=columns, index=df.index)

    ### Missing data flag
    df[columns] = df[columns].fillna(0)
    # Fix: Add new column to main dataframe
    df['missing_financials'] = (df[columns] == 0).sum(axis=1)
    # Note: Removed the .value_counts() as it doesn't assign to anything

    ### Calculate change percentages
    for col in df[columns].columns:
        if col.endswith('_trend'):
            first_col = col.replace('_trend', '_first')
            recent_col = col.replace('_trend', '_recent')
            change_col = col.replace('_trend', '_change')
            
            # Fix: Add new columns to main dataframe
            df[first_col] = df[recent_col] - df[col]
            df[change_col] = safe_ratio(df[col], df[first_col])

    
    # Fix: Add new columns to main dataframe, not subset
    df['asset_to_liability_ratio'] = safe_ratio(
        df['total_assets_recent'], 
        df['total_liabilities_recent']
    )

    df['liability_to_capital_ratio'] = safe_ratio(
        df['total_liabilities_recent'], 
        df['total_capitalization_recent']
    )

    df['asset_to_capital_ratio'] = safe_ratio(
        df['total_assets_recent'], 
        df['total_capitalization_recent']
    )

    df['liability_to_assets_ratio'] = safe_ratio(
        df['total_liabilities_recent'], 
        df['total_assets_recent']
    )

    df['assets_to_public_price_ratio'] = safe_ratio(
        df['total_assets_recent'], 
        df['public_price_per_share']
    )

    df['liabilities_to_public_price_ratio'] = safe_ratio(
        df['total_liabilities_recent'], 
        df['public_price_per_share']
    )

    return df

cleaned_df = prepare_bank_count_feature(original_df, bank_columns)
cleaned_df = prepare_keyword_count_feature(cleaned_df, keyword_columns)
cleaned_df = prepare_financial_feature(cleaned_df, financial_columns)

## Convert Dates to numeric
cleaned_df['ipo_date'] = pd.to_datetime(cleaned_df['ipo_date'])
cleaned_df['ipo_day'] = cleaned_df['ipo_date'].dt.day
cleaned_df['ipo_month'] = cleaned_df['ipo_date'].dt.month
cleaned_df['ipo_year'] = cleaned_df['ipo_date'].dt.year

### Drop String columns
cleaned_df.drop('symbol', axis=1, inplace=True, errors='ignore')
cleaned_df.drop('url', axis=1, inplace=True, errors='ignore')
cleaned_df.drop('ipo_date', axis=1, inplace=True, errors='ignore')
cleaned_df.drop('day', axis=1, inplace=True, errors='ignore')

## Transform to one-hot encoded
mlb_bank = MultiLabelBinarizer()
mlb_keyword = MultiLabelBinarizer()

# Encode bank_group
encoded_bank_group = mlb_bank.fit_transform(cleaned_df['bank_group'])
bank_df = pd.DataFrame(
    encoded_bank_group,
    columns=[f'bank_group_{group}' for group in mlb_bank.classes_],
    index=cleaned_df.index
)

# Encode keyword_group
encoded_keyword_group = mlb_keyword.fit_transform(cleaned_df['keyword_group'])
keyword_df = pd.DataFrame(
    encoded_keyword_group,
    columns=[f'keyword_group_{group}' for group in mlb_keyword.classes_],
    index=cleaned_df.index
)

# Drop original array columns
cleaned_df.drop(['bank_group', 'keyword_group'], axis=1, inplace=True)

# Concatenate all dataframes
cleaned_df = pd.concat([cleaned_df, bank_df, keyword_df], axis=1)

cleaned_df.to_csv('../data/processed_data.csv', index=False)

In [ ]:
df = pd.read_csv('../data/processed_data.csv')
df.head()

,additional_paid_in_capital_trend,additional_paid_in_capital_recent,total_assets_trend,total_assets_recent,total_current_liabilities_trend,total_current_liabilities_recent,total_liabilities_trend,total_liabilities_recent,cash_trend,cash_recent,...,keyword_group_25,keyword_group_26,keyword_group_27,keyword_group_28,keyword_group_29,keyword_group_30,keyword_group_31,keyword_group_32,keyword_group_33,keyword_group_34
0,-0.065148,-0.143178,-0.070095,-0.090368,-0.040564,-0.053970,-0.044359,-0.063151,0.000000,0.000000,...,0,1,0,0,1,0,1,0,0,1
1,-0.024413,-0.106855,-0.069812,-0.090995,-0.039780,-0.055513,-0.044359,-0.063882,-0.044771,-0.077408,...,0,0,0,0,0,0,1,0,0,1
2,-0.024413,-0.106855,-0.069812,-0.090995,-0.039780,-0.055513,-0.044359,-0.063882,-0.044771,-0.077408,...,0,0,0,0,0,0,1,0,0,1
3,-0.064021,-0.142304,-0.069993,-0.091047,-0.040536,-0.055835,-0.044342,-0.064012,-0.045131,-0.077720,...,0,1,1,0,1,1,1,0,1,0
4,-0.065148,-0.106871,-0.070095,-0.091165,-0.040564,-0.055926,-0.044359,-0.038477,0.000000,0.000000,...,0,1,0,0,1,1,1,0,1,1
